In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sodapy import Socrata
from config import app_token
import datetime
import dateutil
import scipy.stats as stats
from citipy import citipy

In [13]:
#!pip install sodapy

In [26]:
#hold cell for tokenized requests
    #unauthenticated client only works with public data sets. Note 'None'
        #in place of application token, and no username or password:
        #client = Socrata("data.lacity.org", None)

#authenticated client request (needed for non-public datasets):
client = Socrata("data.lacity.org",
    app_token=app_token)
    #username=username,
    #password=password)

#first 100,000 results, returned as JSON from API / converted to Python list of
#dictionaries by sodapy
results = client.get("63jg-8b9z", limit=100000)

#convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
#results_df.head()

In [16]:
results_df.columns

Index(['dr_no', 'date_rptd', 'date_occ', 'time_occ', 'area', 'area_name',
       'rpt_dist_no', 'part_1_2', 'crm_cd', 'crm_cd_desc', 'mocodes',
       'vict_age', 'vict_sex', 'vict_descent', 'premis_cd', 'premis_desc',
       'status', 'status_desc', 'crm_cd_1', 'location', 'lat', 'lon',
       'cross_street', 'weapon_used_cd', 'weapon_desc', 'crm_cd_2', 'crm_cd_3',
       'crm_cd_4'],
      dtype='object')

In [39]:
results_df['nearest_city'] = results_df.apply(lambda row: safeCity(row.lat, row.lon), axis=1)

In [30]:
def safeCity(lat, lon):
    try:
        return citipy.nearest_city(float(lat), float(lon)).city_name
    except:
        return 'n/a'

In [42]:
results_df.nearest_city.value_counts()

los angeles             25925
west hollywood          16211
san fernando             9096
beverly hills            7670
culver city              6301
lomita                   6120
glendale                 5372
calabasas                5028
inglewood                4255
south pasadena           3402
santa monica             2855
west carson              2192
westmont                 2169
east los angeles         1981
carson                    489
gardena                   217
huntington park           137
long beach                127
walnut park               122
burbank                   103
pasadena                   61
rancho palos verdes        45
monterey park              23
alhambra                   16
paramount                  13
signal hill                12
santa clarita               5
manhattan beach             5
maywood                     5
torrance                    4
takoradi                    4
downey                      4
willowbrook                 3
el segundo

In [43]:
results_df.head()

,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,...,location,lat,lon,cross_street,weapon_used_cd,weapon_desc,crm_cd_2,crm_cd_3,crm_cd_4,nearest_city
0,001307355,2010-02-20T00:00:00.000,2010-02-20T00:00:00.000,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,300 E GAGE AV,33.9825,-118.2695,NaN,NaN,NaN,NaN,NaN,NaN,huntington park
1,011401303,2010-09-13T00:00:00.000,2010-09-12T00:00:00.000,0045,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,SEPULVEDA BL,33.9599,-118.3962,MANCHESTER AV,NaN,NaN,NaN,NaN,NaN,inglewood
2,070309629,2010-08-09T00:00:00.000,2010-08-09T00:00:00.000,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,1300 E 21ST ST,34.0224,-118.2524,NaN,NaN,NaN,NaN,NaN,NaN,los angeles
3,090631215,2010-01-05T00:00:00.000,2010-01-05T00:00:00.000,0150,06,Hollywood,0646,2,900,VIOLATION OF COURT ORDER,...,CAHUENGA BL,34.1016,-118.3295,HOLLYWOOD BL,102,HAND GUN,998,NaN,NaN,west hollywood
4,100100501,2010-01-03T00:00:00.000,2010-01-02T00:00:00.000,2100,01,Central,0176,1,122,"RAPE, ATTEMPTED",...,8TH ST,34.0387,-118.2488,SAN PEDRO ST,400,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",NaN,NaN,NaN,los angeles


In [44]:
#select only required fields for analysis
subset_df = results_df.loc[:,('nearest_city','area', 'area_name', 'crm_cd', 'crm_cd_desc', 'date_occ', 'date_rptd', 'dr_no', 'lat', 'lon', 'location', 'mocodes', 'premis_cd', 'rpt_dist_no', 'time_occ')]
subset_df.head()
# subset_df.info()
# subset_df.shape

,nearest_city,area,area_name,crm_cd,crm_cd_desc,date_occ,date_rptd,dr_no,lat,lon,location,mocodes,premis_cd,rpt_dist_no,time_occ
0,huntington park,13,Newton,900,VIOLATION OF COURT ORDER,2010-02-20T00:00:00.000,2010-02-20T00:00:00.000,001307355,33.9825,-118.2695,300 E GAGE AV,0913 1814 2000,501,1385,1350
1,inglewood,14,Pacific,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",2010-09-12T00:00:00.000,2010-09-13T00:00:00.000,011401303,33.9599,-118.3962,SEPULVEDA BL,0329,101,1485,0045
2,los angeles,13,Newton,946,OTHER MISCELLANEOUS CRIME,2010-08-09T00:00:00.000,2010-08-09T00:00:00.000,070309629,34.0224,-118.2524,1300 E 21ST ST,0344,103,1324,1515
3,west hollywood,06,Hollywood,900,VIOLATION OF COURT ORDER,2010-01-05T00:00:00.000,2010-01-05T00:00:00.000,090631215,34.1016,-118.3295,CAHUENGA BL,1100 0400 1402,101,0646,0150
4,los angeles,01,Central,122,"RAPE, ATTEMPTED",2010-01-02T00:00:00.000,2010-01-03T00:00:00.000,100100501,34.0387,-118.2488,8TH ST,0400,103,0176,2100


In [ ]:
#rename columns
subset_df.rename(
  columns={
    #0 : 'title',
    #1 : 'author'
    'nearst_city: 'City
    'area': 'Area ID', 
    'area_name': 'Area Name',
    'crm_cd': 'Crime Code',
    'crm_cd_desc': 'Crime Code Description',
    'date_occ': 'Date Occurred',
    'date_rptd': 'Date Reported',
    'dr_no': 'DR #',
    'lat': 'Latitude',
    'lon': 'Longitude',
    'location': 'Location',
    'mocodes': 'MO Code',
    'premis_cd': 'Premise Code',
    'rpt_dist_no': 'Reporting District',
    'time_occ': 'Time Occurred'
  },
  inplace=True
)
subset_df